<a href="https://colab.research.google.com/github/yucucan/DSA-210-Term-Project/blob/main/Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [358]:
from os.path import join
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# to be used a prefix for file I/O
path_prefix = "/content/drive/My Drive/Colab/DSA 210/Project/Data"

Mounted at /content/drive


In [359]:
import os
print(len(os.listdir(path_prefix)))  # path_prefix as you provided

56


In [360]:
worldbank_fname_list = ["Agricultural land (% of land area)",
              "Forest area (% of land area)",
              "Rural population (% of total population)",
              "Agriculture, forestry, and fishing, value added (% of GDP)",
              "Maternal mortality ratio (modeled estimate, per 100,000 live births)",
              "Mortality rate, under-5 (per 1,000 live births)",
              "Net migration",
              "Access to electricity (% of population)",
              "Electric power consumption (kWh per capita)",
              "Population growth (annual %)",
              "Urban population (% of total population)",
              "GDP per capita (current US$)",
              "GDP per capita growth (annual %)",
              "GNI per capita, PPP (current international $)",
              "Inflation, consumer prices (annual %)",
              "Government expenditure on education, total (% of GDP)",
              "Time required to get electricity (days)",
              "Renewable internal freshwater resources per capita (cubic meters)",
              "Renewable electricity output (% of total electricity output)",
              "Strength of legal rights index (0=weak to 12=strong)",
              "Labor force, female (% of total labor force)",
              "Life expectancy at birth, female (years)",
              "Life expectancy at birth, male (years)",
              "Fertility rate, total (births per woman)",
              "Death rate, crude (per 1,000 people)",
              "Fixed broadband subscriptions (per 100 people)",
              "Mobile cellular subscriptions (per 100 people)",
              "Secure Internet servers (per 1 million people)",
              "High-technology exports (current US$)",
              "Logistics performance index Overall (1=low to 5=high)",
              "Medium and high-tech exports (% manufactured exports)",
              "New businesses registered (number)",
              "Population, total",
              "Military expenditure (% of GDP)",
              "Tax revenue (% of GDP)",
              "Unemployment, total (% of total labor force) (modeled ILO estimate)",
              "Employment to population ratio, 15+, total (%) (modeled ILO estimate)",
              "Exports of goods and services (% of GDP)",
              "PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)",
              "Population density (people per sq. km of land area)",
              "Population in the largest city (% of urban population)",
              "Refugee population by country or territory of asylum"
              ]
exceptions_2015 = ["Renewable electricity output (% of total electricity output).csv",]
exceptions_2014 = ["Electric power consumption (kWh per capita).csv"]

In [361]:
worldbank_fname_list_csv = worldbank_fname_list.copy()
for i in range(len(worldbank_fname_list)):
  worldbank_fname_list_csv[i] = worldbank_fname_list[i] + ".csv"

In [362]:
# Datas with mising values over 50
n = os.listdir(path_prefix)
for item in n:
  if item not in worldbank_fname_list_csv:
    print(item)

Literacy rate, adult total (% of people ages 15 and above).csv
Central government debt, total (% of GDP).csv
Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population).csv
Income share held by lowest 20%.csv
CPIA public sector management and institutions cluster average (1=low to 6=high).csv
Population in urban agglomerations of more than 1 million (% of total population).csv
School enrollment, primary and secondary (gross), gender parity index (GPI).csv
Literacy rate, youth total (% of people ages 15-24).csv
Firms with female participation in ownership (% of firms).csv
Cause of death, by injury (% of total).csv
Income share held by highest 20%.csv
Contraceptive prevalence, any method (% of married women ages 15-49).csv
Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant).csv
Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol), three-year average.csv


In [363]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [364]:
### Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pickle
import warnings
warnings.filterwarnings(action='ignore')

# Data Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Model Training
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

## Import necessary modules for content-based filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [365]:
!kaggle datasets download -d mathurinache/world-happiness-report

!unzip "world-happiness-report.zip"

Dataset URL: https://www.kaggle.com/datasets/mathurinache/world-happiness-report
License(s): CC0-1.0
world-happiness-report.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  world-happiness-report.zip
replace 2015.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Ay
  inflating: 2015.csv                
  inflating: 2016.csv                
  inflating: 2017.csv                
  inflating: 2018.csv                
  inflating: 2019.csv                
  inflating: 2020.csv                
  inflating: 2021.csv                
  inflating: 2022.csv                


In [366]:
!kaggle datasets download -d azathoth42/myanimelist

!unzip "myanimelist.zip"

Dataset URL: https://www.kaggle.com/datasets/azathoth42/myanimelist
License(s): DbCL-1.0
myanimelist.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  myanimelist.zip
replace AnimeList.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Ay
  inflating: AnimeList.csv           
  inflating: UserAnimeList.csv       y
y

  inflating: UserList.csv            
  inflating: anime_cleaned.csv       
  inflating: anime_filtered.csv      
  inflating: animelists_cleaned.csv  y

  inflating: animelists_filtered.csv  y

  inflating: users_cleaned.csv       
  inflating: users_filtered.csv      


In [367]:
df = pd.read_csv('2016.csv')
df.sort_values(by='Country', ascending=True, inplace=True)
df = df[df["Country"] != "Kosovo"]
df.reset_index(drop=True, inplace=True)
df['Country'] = df['Country'].str.lower()
df['Country'] = df['Country'].str.strip()

In [368]:
rows_to_skip = 4
for i in range (len(worldbank_fname_list_csv)):
  fname = worldbank_fname_list_csv[i]
  df_world_bank = pd.read_csv(join(path_prefix, fname), skiprows=rows_to_skip, sep=',', on_bad_lines='skip')
  df_world_bank['Country Name'] = df_world_bank['Country Name'].str.lower().str.strip()
  df_tmp = df[['Country']].copy()
  if fname in exceptions_2015:
    df_tmp = df_tmp.merge(df_world_bank[['Country Name', '2015']], left_on='Country', right_on='Country Name', how='left')
    df[worldbank_fname_list[i]] = df_tmp['2015']
    break
  elif fname in exceptions_2014:
    df_tmp = df_tmp.merge(df_world_bank[['Country Name', '2014']], left_on='Country', right_on='Country Name', how='left')
    df[worldbank_fname_list[i]] = df_tmp['2014']
  else:
    df_tmp = df_tmp.merge(df_world_bank[['Country Name', '2016']], left_on='Country', right_on='Country Name', how='left')
    df[worldbank_fname_list[i]] = df_tmp['2016']
df

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,...,Population growth (annual %),Urban population (% of total population),GDP per capita (current US$),GDP per capita growth (annual %),"GNI per capita, PPP (current international $)","Inflation, consumer prices (annual %)","Government expenditure on education, total (% of GDP)",Time required to get electricity (days),Renewable internal freshwater resources per capita (cubic meters),Renewable electricity output (% of total electricity output)
0,afghanistan,Southern Asia,154,3.360,3.288,3.432,0.38227,0.11037,0.17344,0.16430,...,2.535720,25.020,522.082216,-0.300121,2240.0,4.383892,4.543970,114.0,1358.765661,86.050111
1,albania,Central and Eastern Europe,109,4.655,4.546,4.764,0.95530,0.50163,0.73007,0.31866,...,-0.159880,58.421,4124.055390,3.480293,12270.0,1.275432,3.962090,134.0,9352.939970,100.000000
2,algeria,Middle East and Northern Africa,38,6.355,6.227,6.483,1.05266,0.83309,0.61804,0.21006,...,2.055691,71.459,4424.985290,1.785941,13350.0,6.397695,6.502109,180.0,275.319498,0.322684
3,angola,Sub-Saharan Africa,141,3.866,3.753,3.979,0.84731,0.66366,0.04991,0.00589,...,3.576441,64.149,1807.952941,-6.002649,6490.0,30.694415,2.754937,145.0,5071.433540,53.174928
4,argentina,Latin America and Caribbean,26,6.650,6.560,6.740,1.15137,1.06612,0.69711,0.42284,...,0.968911,91.627,12699.962314,-3.024501,19660.0,NaN,5.545490,92.0,6651.433214,28.140069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,venezuela,Latin America and Caribbean,44,6.084,5.973,6.195,1.13367,1.03302,0.61904,0.19847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,vietnam,Southeastern Asia,96,5.061,4.991,5.131,0.74037,0.79117,0.66157,0.55954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,yemen,Middle East and Northern Africa,147,3.724,3.621,3.827,0.57939,0.47493,0.31048,0.22870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,zambia,Sub-Saharan Africa,106,4.795,4.645,4.945,0.61202,0.63760,0.23573,0.42662,...,3.094091,42.438,1239.085279,0.593980,3210.0,17.869730,3.747920,117.0,4741.515569,96.993824


In [369]:
df_world_bank.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,aruba,ABW,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,14.856161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,africa eastern and southern,AFE,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,25.675879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,afghanistan,AFG,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.050111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,africa western and central,AFW,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.233531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,angola,AGO,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,53.174928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [370]:
for col in df.columns:
    if df[col].isnull().any():
        nan_sum = df[col].isnull().sum()
        print(f"Column '{col}' has {nan_sum} NaN values.")

Column 'Agricultural land (% of land area)' has 22 NaN values.
Column 'Forest area (% of land area)' has 22 NaN values.
Column 'Rural population (% of total population)' has 22 NaN values.
Column 'Agriculture, forestry, and fishing, value added (% of GDP)' has 24 NaN values.
Column 'Maternal mortality ratio (modeled estimate, per 100,000 live births)' has 22 NaN values.
Column 'Mortality rate, under-5 (per 1,000 live births)' has 23 NaN values.
Column 'Net migration' has 22 NaN values.
Column 'Access to electricity (% of population)' has 22 NaN values.
Column 'Electric power consumption (kWh per capita)' has 40 NaN values.
Column 'Population growth (annual %)' has 22 NaN values.
Column 'Urban population (% of total population)' has 22 NaN values.
Column 'GDP per capita (current US$)' has 23 NaN values.
Column 'GDP per capita growth (annual %)' has 23 NaN values.
Column 'GNI per capita, PPP (current international $)' has 23 NaN values.
Column 'Inflation, consumer prices (annual %)' has 

In [371]:
country_list = df['Country'].tolist()

In [387]:
import datetime

users = pd.read_csv('users_cleaned.csv')
users['join_date'] = pd.to_datetime(users['join_date'])
users['last_online'] = pd.to_datetime(users['last_online'])
users['birth_date'] = pd.to_datetime(users['birth_date'])
now = datetime.datetime.now()
users['age'] = (now - users['birth_date']).dt.days / 365.25
users['age'] = users['age'].astype(int)
users

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,age
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391,34
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01,NaN,2008-02-13,2017-07-10 06:52:54,6.15,6.0,4903,33
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14,NaN,2009-08-31,2014-05-12 16:35:00,8.27,1.0,9701,34
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24,NaN,2010-05-10,2012-10-18 19:06:00,9.70,6.0,697,26
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24,NaN,2015-11-26,2018-05-10 20:53:37,7.86,0.0,1847,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13,NaN,2011-10-23,2018-02-25 01:20:27,8.03,2.0,3939,28
108707,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18,NaN,2012-07-14,2017-02-14 00:30:00,8.86,24.0,2537,26
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17,NaN,2013-04-24,2016-12-18 08:41:00,7.40,1.0,2869,29
108709,Torasori,3975907,22,239,0,4,176,72.361111,Male,"Latvia, Riga",1998-11-18,NaN,2014-07-30,2018-05-24 21:34:46,8.98,47.0,4469,26


In [388]:
users = users.drop(columns=['access_rank'])

In [389]:
us_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
    "New Hampshire", "New Jersey", "New Mexico", "New York",
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]
us_states_lower = [state.lower() for state in us_states]

In [390]:
def find_country(input):
  input = str(input)
  if input == 'nan':
    return 'Other'
  else:
    for country in country_list:
      if country in input.lower():
        return country
    for state in us_states_lower:
      if state in input.lower():
        return 'united states'
    return 'Other'

In [391]:
users['Country'] = users['location'].apply(find_country)
users

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,age,Country
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391,34,india
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01,2008-02-13,2017-07-10 06:52:54,6.15,6.0,4903,33,united states
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14,2009-08-31,2014-05-12 16:35:00,8.27,1.0,9701,34,mexico
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24,2010-05-10,2012-10-18 19:06:00,9.70,6.0,697,26,malaysia
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24,2015-11-26,2018-05-10 20:53:37,7.86,0.0,1847,25,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13,2011-10-23,2018-02-25 01:20:27,8.03,2.0,3939,28,poland
108707,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18,2012-07-14,2017-02-14 00:30:00,8.86,24.0,2537,26,united states
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17,2013-04-24,2016-12-18 08:41:00,7.40,1.0,2869,29,belgium
108709,Torasori,3975907,22,239,0,4,176,72.361111,Male,"Latvia, Riga",1998-11-18,2014-07-30,2018-05-24 21:34:46,8.98,47.0,4469,26,latvia


In [392]:
other_count = users[users['Country'] == 'Other'].shape[0]
print(f"Number of 'Other' values in the 'Country' column: {other_count}")

Number of 'Other' values in the 'Country' column: 40555


In [393]:
users = users[users['Country'] != 'Other']
users.reset_index(drop=True, inplace=True)
users

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,age,Country
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391,34,india
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01,2008-02-13,2017-07-10 06:52:54,6.15,6.0,4903,33,united states
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14,2009-08-31,2014-05-12 16:35:00,8.27,1.0,9701,34,mexico
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24,2010-05-10,2012-10-18 19:06:00,9.70,6.0,697,26,malaysia
4,HimeAria,3129315,2,87,2,0,28,19.963194,Female,Poland,1996-09-26,2013-09-08,2018-01-14 13:05:42,8.23,0.0,1314,28,poland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68151,ZeroFr3aK,472691,36,179,12,0,38,125.084028,Male,Sweden :3,1993-02-14,2011-04-11,2018-04-09 18:01:39,7.41,6.0,7598,32,sweden
68152,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13,2011-10-23,2018-02-25 01:20:27,8.03,2.0,3939,28,poland
68153,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18,2012-07-14,2017-02-14 00:30:00,8.86,24.0,2537,26,united states
68154,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17,2013-04-24,2016-12-18 08:41:00,7.40,1.0,2869,29,belgium


In [394]:
final_df = pd.merge(users, df, on='Country', how='inner')
final_df

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,...,Population growth (annual %),Urban population (% of total population),GDP per capita (current US$),GDP per capita growth (annual %),"GNI per capita, PPP (current international $)","Inflation, consumer prices (annual %)","Government expenditure on education, total (% of GDP)",Time required to get electricity (days),Renewable internal freshwater resources per capita (cubic meters),Renewable electricity output (% of total electricity output)
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",...,1.191630,33.182,1707.508929,6.973947,5730.0,4.948216,4.256516,59.21,1075.937451,15.343050
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",...,0.724676,81.862,58206.614193,1.084257,58760.0,1.261583,4.783280,89.60,8722.520482,13.228593
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",...,0.969124,79.577,9097.923977,0.790955,19580.0,2.821708,4.757960,100.44,3345.566300,15.394134
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",...,1.767311,74.840,9476.534098,2.620046,24590.0,2.090567,4.345984,31.00,18244.911832,9.957168
4,HimeAria,3129315,2,87,2,0,28,19.963194,Female,Poland,...,-0.042985,60.178,12464.011051,3.075475,27250.0,-0.664767,4.659110,122.00,1411.637535,13.802399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68151,ZeroFr3aK,472691,36,179,12,0,38,125.084028,Male,Sweden :3,...,1.256454,86.852,51820.397248,1.071809,50420.0,0.984269,7.618620,52.00,17232.544113,63.262748
68152,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,...,-0.042985,60.178,12464.011051,3.075475,27250.0,-0.664767,4.659110,122.00,1411.637535,13.802399
68153,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",...,0.724676,81.862,58206.614193,1.084257,58760.0,1.261583,4.783280,89.60,8722.520482,13.228593
68154,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,...,0.506300,97.919,41854.549830,0.682455,49060.0,1.973853,6.461730,201.00,1059.002127,20.800023


In [395]:
for col in final_df.columns:
    if final_df[col].isnull().any():
        nan_sum = final_df[col].isnull().sum()
        print(f"Column '{col}' has {nan_sum} NaN values.")

Column 'username' has 1 NaN values.
Column 'Agricultural land (% of land area)' has 4582 NaN values.
Column 'Forest area (% of land area)' has 4582 NaN values.
Column 'Rural population (% of total population)' has 4582 NaN values.
Column 'Agriculture, forestry, and fishing, value added (% of GDP)' has 4582 NaN values.
Column 'Maternal mortality ratio (modeled estimate, per 100,000 live births)' has 4582 NaN values.
Column 'Mortality rate, under-5 (per 1,000 live births)' has 4729 NaN values.
Column 'Net migration' has 4582 NaN values.
Column 'Access to electricity (% of population)' has 4582 NaN values.
Column 'Electric power consumption (kWh per capita)' has 4757 NaN values.
Column 'Population growth (annual %)' has 4582 NaN values.
Column 'Urban population (% of total population)' has 4582 NaN values.
Column 'GDP per capita (current US$)' has 4582 NaN values.
Column 'GDP per capita growth (annual %)' has 4582 NaN values.
Column 'GNI per capita, PPP (current international $)' has 4582